In [3]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import os
import sys

import plotly.graph_objects as go
import plotly.express as px
import pandas as pd


import pandas as pd
import plotly.express as px
import plotly.graph_objs as go

sys.path.insert(0, os.path.abspath('../developer'))

from config import MOCK_DATA, CODE, OUT, IN
from developer.utilities import read_yaml
from analysis.model import load_model

In [3]:
model = load_model(OUT / "models" / "model_amce_high_trust.pickle")

In [5]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                support   R-squared:                       0.028
Model:                            OLS   Adj. R-squared:                  0.026
Method:                 Least Squares   F-statistic:                     520.1
Date:                Thu, 14 Sep 2023   Prob (F-statistic):               0.00
Time:                        17:48:56   Log-Likelihood:                -7507.9
No. Observations:               12780   AIC:                         1.507e+04
Df Residuals:                   12755   BIC:                         1.525e+04
Df Model:                          24                                         
Covariance Type:              cluster                                         
=============================================================================================
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                         0.4273      0.027     15.579      0.000       0.374       0.481
att_1_Eliminate2050           0.0694      0.010      6.904      0.000       0.050       0.089
att_1_Reduce2030              0.0850      0.011      7.732      0.000       0.063       0.107
att_2_EnergyAccess            0.0752      0.011      6.843      0.000       0.054       0.097
att_2_InsureWorkers           0.0791      0.011      7.017      0.000       0.057       0.101
att_2_LowPrices               0.0773      0.011      7.047      0.000       0.056       0.099
att_3_IdentityCoalRegions     0.0430      0.012      3.703      0.000       0.020       0.066
att_3_IndustryCoalRegions     0.0633      0.011      5.509      0.000       0.041       0.086
att_3_WorkersCoalRegion       0.0624      0.012      5.361      0.000       0.040       0.085
att_4_Gov&Businesses          0.0299      0.014      2.157      0.031       0.003       0.057
att_4_Gov&CivilSociety        0.0425      0.014      3.065      0.002       0.015       0.070
att_4_Gov&LaborUnions         0.0319      0.014      2.265      0.024       0.004       0.060
att_4_Gov&LocalGov            0.0563      0.014      4.075      0.000       0.029       0.083
att_4_Gov&Researchers         0.0364      0.014      2.600      0.009       0.009       0.064
att_5_Fin&TechSupport         0.0803      0.012      6.660      0.000       0.057       0.104
att_5_FinSupportOnly          0.0157      0.012      1.340      0.180      -0.007       0.039
att_5_TechSupportOnly         0.0162      0.012      1.384      0.166      -0.007       0.039
ageFilter                     0.0004      0.001      0.654      0.513      -0.001       0.002
genderFilter                  0.0121      0.015      0.818      0.413      -0.017       0.041
district_NorthernZone         0.0887      0.015      5.967      0.000       0.060       0.118
district_NorthEasternZone     0.0339      0.033      1.019      0.308      -0.031       0.099
district_CentralZone          0.0576      0.024      2.398      0.016       0.011       0.105
district_EasternZone          0.0846      0.016      5.197      0.000       0.053       0.116
district_WesternZone          0.1243      0.018      6.850      0.000       0.089       0.160
district_SouthernZone         0.0381      0.017      2.271      0.023       0.005       0.071
treatment_status             -0.0098      0.015     -0.669      0.503      -0.038       0.019
==============================================================================
Omnibus:                     2756.601   Durbin-Watson:                   1.434
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2566.791
Skew:                          -1.018   Prob(JB):                         0.00
Kurtosis:                       2.178   C

### Overall support

In [11]:
conjoint = pd.read_csv(OUT / "data" / "data_long.csv")

treated = conjoint[conjoint['treatment_status'] == 1]
control = conjoint[conjoint['treatment_status'] == 0]
labels = ["Treated", "Control"]

avg_support_treated = treated['support'].astype(int).mean()
std_support_treated = treated['support'].astype(int).std()
nobs_support_treated = len(treated['support'])

avg_support_control = control['support'].astype(int).mean()
std_support_control = control['support'].astype(int).std()
nobs_support_control = len(control['support'])

values = [avg_support_treated, avg_support_control]
std_dev = [std_support_treated, std_support_control]
labels = ["Treated", "Control"]

# Calculate confidence intervals (assuming a normal distribution)
confidence_intervals_treated = [(avg_support_treated - 1.96 * (std_support_treated / (nobs_support_treated) ** 0.5)), avg_support_treated + 1.96 * (std_support_treated / (nobs_support_treated) ** 0.5)]
confidence_intervals_control = [(avg_support_control - 1.96 * (std_support_control / (nobs_support_treated) ** 0.5)), avg_support_control + 1.96 * (std_support_control / (nobs_support_control) ** 0.5)]


# Create a bar plot with error bars representing confidence intervals
fig = go.Figure()

fig.add_trace(go.Bar(
    x=labels,
    y=values,
    error_y=dict(
                type='data',
                array=[max(confidence_intervals_treated) - values[0], max(confidence_intervals_control) - values[1]],
                arrayminus=[values[0] - min(confidence_intervals_treated), values[1] - min(confidence_intervals_control)],
                visible=True
            ),
    name='Values',
    marker_color=['red', 'blue'],
    width=0.6
))

fig.update_layout(yaxis_range=[0, 1], xaxis_range=[-1,2], width=400, height=500)
# Customize the layout
fig.update_layout(
    title='Fig C: General support',
    xaxis=dict(title='Values'),
    yaxis=dict(title='Mean'),
    barmode='group',
    bargap=0.1
    
)

# Show the plot
fig.show()

##### Regions

In [12]:
conjoint = pd.read_csv(OUT / "data" / "data_clean.csv")

conjoint["locationFilter"].unique()

array(['Puducherry', 'Dadra and Nagar Haveli and Daman & Diu',
       'Uttar Pradesh', 'Ladakh', 'West Bengal', 'Tripura'], dtype=object)

#### Frequencies of attributes

In [13]:
conjoint_reg = pd.read_csv(OUT / "data" / "data_regression.csv")

#conjoint_dummy.columns.str.startswith('att')
def frequencies(conjoint_reg):
    
    frequency_table =  {}
    groups = ['att_1', 'att_2','att_3', 'att_4', 'att_5'] #add 6
    for group in groups:
        total = sum(dict(conjoint_reg.filter(like=group).sum()).values())
        frequency_table[group] = {key.replace(f'{group}_', '') : (value / total).round(2) for key, value in dict(conjoint_reg.filter(like=group).sum()).items()}
    frequency_table = pd.DataFrame(frequency_table).sum(axis=1)
    frequency_table = pd.DataFrame(frequency_table, columns=["frequency"])

    frequency_table = frequency_table.rename_axis("Attribute_level")
    return frequency_table

frequencies(conjoint_reg)

,frequency
Attribute_level,
Eliminate2050,0.34
Eliminate2070,0.33
Reduce2030,0.33
EnergyAccess,0.25
InsureWorkers,0.25
LowPrices,0.25
NothingSoc,0.25
IdentityCoalRegions,0.25
IndustryCoalRegions,0.25


#### Time spent on surveys

In [6]:

import plotly.express as px
import plotly.figure_factory as ff


clean_data = pd.read_csv(OUT / "data" / "data_clean.csv")

clean_data = clean_data[clean_data['aware'] == 0]

clean_data['DurationMin'] = clean_data['Duration (in seconds)'] / 60

clean_data = clean_data[clean_data['DurationMin'] < 60]

# Calculate the average and median
average_duration = clean_data['DurationMin'].mean()
median_duration = clean_data['DurationMin'].median()

fig = ff.create_distplot([clean_data['DurationMin']], ['DurationMin'], show_curve=True, show_rug=False, bin_size=2.0)

fig.add_shape(
    type='line',
    x0=average_duration,
    x1=average_duration,
    y0=0,
    y1=1,
    xref='x',
    yref='paper',
    line=dict(color='red', width=2, dash='dash'),
    name=f'Average: {average_duration:.2f}'
)

fig.add_shape(
    type='line',
    x0=median_duration,
    x1=median_duration,
    y0=0,
    y1=1,
    xref='x',
    yref='paper',
    line=dict(color='green', width=2, dash='dash'),
    name=f'Median: {median_duration:.2f}'
)

# Add labels for average and median values
fig.add_annotation(
    x=average_duration,
    y=1.08,
    xref='x',
    yref='paper',
    text=f'Average: {average_duration:.2f}',
    showarrow=False,
    arrowhead=4,
    ax=0,
    ay=-40
)

fig.add_annotation(
    x=median_duration,
    y=1.13,
    xref='x',
    yref='paper',
    text=f'Median: {median_duration:.2f}',
    showarrow=False,
    arrowhead=4,
    ax=0,
    ay=-40
)

# Update the layout if needed
fig.update_layout(
    title='Density Plot with Smooth Line',
    xaxis_title='Value',
    yaxis_title='Density'
)
# Update the layout if needed
fig.update_layout(title='Fig B: Density plot of time spent for engaged', xaxis_title='Value', yaxis_title='Density')


# Show the plot
fig.show()

#### Word count

In [23]:
clean_data = pd.read_csv(IN / "main_sample.csv")

clean_data = clean_data.drop([0, 1])

clean_data["opinion_coal_ind"]
clean_data["feedback"] = clean_data["feedback"].fillna(' ')


average_char = clean_data["feedback"].str.len().mean()

average_words = clean_data["feedback"].str.split().apply(len).mean()


average_words

9.42580164056674

#### Treatment

In [33]:
import pandas as pd

def attribute_support(df, attribute, categories, att_nr, group1, group2, dummy,extra=""):
    df = df.copy()

    df_control = df[df[dummy] == 0]
    df_treated = df[df[dummy] == 1]

    df_treated = df_treated[[attribute, 'support']]
    df_treated['support'] = df_treated['support'].astype(int)

    support_control = {"Attribute Level": [], "Value": [], "CI_lower": [], "CI_upper": []}
    
    for cat in categories[::-1]:
        group = df_control[df_control[attribute] == cat]
        mean = group['support'].mean()
        std_dev = group['support'].std()
        n = len(group)
        confidence_interval = 1.96 * (std_dev / (n**0.5))  # 95% confidence interval

        support_control["Attribute Level"].append(cat.replace('&', '<br>'))
        support_control["Value"].append(mean.round(2))
        support_control["CI_lower"].append((mean - confidence_interval).round(2))
        support_control["CI_upper"].append((mean + confidence_interval).round(2))

    df_control = pd.DataFrame(support_control)

    support_treated = {"Attribute Level": [], "Value": [], "CI_lower": [], "CI_upper": []}
    
    for cat in categories[::-1]:
        group = df_treated[df_treated[attribute] == cat]
        mean = group['support'].mean()
        std_dev = group['support'].std()
        n = len(group)
        confidence_interval = 1.96 * (std_dev / (n**0.5))  # 95% confidence interval

        support_treated["Attribute Level"].append(cat.replace('&', '<br>'))
        support_treated["Value"].append(mean.round(2))
        support_treated["CI_lower"].append((mean - confidence_interval).round(2))
        support_treated["CI_upper"].append((mean + confidence_interval).round(2))

    df_treated = pd.DataFrame(support_treated)

    color_scale = ["rgb(173, 221, 142)", "rgb(127, 188, 65)", "rgb(78, 139, 37)", "rgb(45, 82, 21)", "rgb(45, 82, 21)", "rgb(45, 82, 21)","rgb(45, 82, 21)"]

    fig = go.Figure()

    for i, row in df_control.iterrows():
        fig.add_trace(go.Bar(
            x=[row["Attribute Level"]],
            y=[row["Value"]],
            error_y=dict(
                type='data',
                array=[row["CI_upper"] - row["Value"]],
                arrayminus=[row["Value"] - row["CI_lower"]],
                visible=True
            ),
            marker_color=color_scale[i],
            name=row["Attribute Level"],
            width=0.2
        ))

    color_scale = ["rgb(221, 142, 142)", "rgb(188, 65, 65)", "rgb(139, 37, 37)", "rgb(82, 21, 21)", "rgb(82, 21, 21)", "rgb(82, 21, 21)", "rgb(82, 21, 21)"]


    for i, row in df_treated.iterrows():
        fig.add_trace(go.Bar(
            x=[row["Attribute Level"]],
            y=[row["Value"]],
            error_y=dict(
                type='data',
                array=[row["CI_upper"] - row["Value"]],
                arrayminus=[row["Value"] - row["CI_lower"]],
                visible=True
            ),
            marker_color=color_scale[i],
            name=row["Attribute Level"],
            width=0.2
        ))

    

    # Set y-axis range from 0 to 1
    fig.update_layout(yaxis_range=[0, 1], xaxis_range=[-1,6], width=700, height=500)

    # Add a horizontal line at y=0.5
    fig.add_hline(y=0.5, line_dash="dash")


    fig.update_layout(barmode="group", bargap=0.6)
    
    fig.update_layout(
        title={
            'text': f"Fig {att_nr}: Att {att_nr} {extra}",
            'x': 0.5,
            'xanchor': 'center',
            'font': {'family': 'Computer Modern'}
        },
        margin=dict(l=20, r=20, t=45, b=5),
        paper_bgcolor="#EADDCA",
        plot_bgcolor='rgba(0,0,0,0)',
        showlegend=False,  # Show legend for different Attribute Levels
        xaxis_showticklabels=True,
        xaxis_title=None,
    )

    fig.add_annotation(
            x=1.5,
            y=0.95,
            xref='x',
            yref='paper',
            text=f'{group2}',
            showarrow=False,
            arrowhead=4,
            font=dict(
                    size=14,
                    color="rgb(139, 37, 37)"
                ),
            ax=0,
            ay=-40
        )
    
    fig.add_annotation(
            x=1.5,
            y=0.9,
            xref='x',
            yref='paper',
            text=f'{group1}',
            showarrow=False,
            arrowhead=4,
            font=dict(
                    size=14,
                    color="rgb(78, 139, 37)"
                ),
            ax=0,
            ay=-40
        )

    return fig

data_clean = pd.read_csv(OUT / "data" / "data_long.csv")

categories = [['Reduce2030','Eliminate2050', 'Eliminate2070'], ['LowPrices', 'InsureWorkers', 'EnergyAccess', 'NothingSoc'], ['IdentityCoalRegions', 'IndustryCoalRegions', 'WorkersCoalRegion', 'NothingEco'],['Gov&CivilSociety', 'Gov&Researchers', 'Gov&Businesses', 'Gov&LaborUnions', 'Gov&LocalGov', 'GovAlone'],['Fin&TechSupport', 'FinSupportOnly', 'TechSupportOnly', 'NoInterference']]

group = "Treatment"

for i in range(1,6):
    fig = attribute_support(data_clean, f"att_{i}", categories[i-1], f"{i}", "Control", "Treated", dummy="treatment_status",extra='Treatment')
    fig.write_image(OUT / "other_fig" / f"fig_{i}_att_{i}_{group}.png")

#### Rating of justice principles


In [5]:
clean_data = pd.read_csv(OUT / "data" / "data_clean.csv")

clean_data = clean_data[clean_data['coal_state'] == 1]

dictionary = {'justice_1' : 'Procedural' , 'justice_2' : 'Distributive', 'justice_3' : 'Recognitional', 'justice_8' : 'Restorative'}
new_cols = ['Procedural' , 'Distributive', 'Recognitional', 'Restorative']

clean_data = clean_data[['justice_1', 'justice_2', 'justice_3', 'justice_8']]

clean_data[new_cols] = clean_data[['justice_1', 'justice_2', 'justice_3', 'justice_8']]

clean_data = clean_data.dropna()

clean_data = clean_data[new_cols].astype(int)

most_important_cols = []

for col in new_cols:
    clean_data[f'{col}_most_important'] = clean_data[col] == 1
    most_important_cols.append(f'{col}_most_important')

means = clean_data[new_cols].astype(int).mean().round(2)
std = clean_data[new_cols].astype(int).std()

most_important = clean_data[most_important_cols].mean()

least_important_cols = []

for col in new_cols:
    clean_data[f'{col}_least_important'] = clean_data[col] == 4
    least_important_cols.append(f'{col}_least_important')

least_important = clean_data[least_important_cols].mean()

most_important


Procedural_most_important       0.218750
Distributive_most_important     0.243750
Recognitional_most_important    0.278125
Restorative_most_important      0.259375
dtype: float64

Awareness in an not in coal state

In [32]:
clean_data = pd.read_csv(OUT / "data" / "data_clean.csv")

clean_data = clean_data[clean_data['coal_state'] == 0]

tot = len(clean_data)

clean_data = clean_data[clean_data['trust_ID'] == 0]

len(clean_data)/tot


0.25219941348973607

### Descriptives

In [5]:
import pandas as pd
from tabulate import tabulate

def generate_descriptive_stats_table(dataframe, columns):
    # Create an empty list to store the rows
    rows = []

    # Iterate through the columns
    for column_name in columns:
        column_data = dataframe[column_name]
        
        if pd.api.types.is_numeric_dtype(column_data):
            # For numeric columns, calculate the average and N
            average = column_data.mean()
            n = column_data.count()
            
            rows.append([column_name, f'{average:.2f}', n])
        elif pd.api.types.is_categorical_dtype(column_data):
            # For categorical columns, calculate percentage shares and N for each category
            category_counts = column_data.value_counts()
            n_total = len(column_data)
            
            for category, count in category_counts.items():
                percentage_share = (count / n_total) * 100
                rows.append([category, f'{percentage_share:.2f}\%', count])
    
    # Create a DataFrame from the list of rows
    result_df = pd.DataFrame(rows, columns=['Category', 'Average', 'N'])
    
    # Convert the result to LaTeX format
    latex_table = result_df.to_latex(index=False, column_format='lcc', escape=False)
    
    return latex_table

df = pd.read_csv(OUT / "data" / "data_clean.csv")

age_bins = [18, 25, 35, 45, 55, 65]
age_labels = ['18-24', '25-34', '35-44', '45-54', '55-64']

# Use the pd.cut() function to create the categorical column
df['age_group'] = pd.cut(df['ageFilter'], bins=age_bins, labels=age_labels, right=False)

#df['ageFilter'] = df['ageFilter'].astype('category')
df['genderFilter'] = df['genderFilter'].astype('category')
df['district'] = df['district'].astype('category')
df['coal_state'] = df['coal_state'].replace({1:'CoalState', 0:'NonCoalState'}).astype('category')

columns_to_analyze = ["age_group", "genderFilter", "district", "coal_state"]
latex_output = generate_descriptive_stats_table(df, columns_to_analyze)
print(latex_output)

\begin{tabular}{lcc}
\toprule
Category & Average & N \\
\midrule
25-34 & 27.67\% & 386 \\
35-44 & 25.02\% & 349 \\
18-24 & 19.21\% & 268 \\
45-54 & 18.71\% & 261 \\
55-64 & 9.18\% & 128 \\
0.000000 & 53.76\% & 750 \\
1.000000 & 46.24\% & 645 \\
NorthernZone & 24.59\% & 343 \\
SouthernZone & 24.52\% & 342 \\
EasternZone & 22.08\% & 308 \\
WesternZone & 16.42\% & 229 \\
CentralZone & 8.39\% & 117 \\
NorthEasternZone & 4.01\% & 56 \\
CoalState & 51.11\% & 713 \\
NonCoalState & 48.89\% & 682 \\
\bottomrule
\end{tabular}



### Income Distribution

### Proximity to coal plants ( ‘haversine’ formula )

In [3]:
import math

data_clean = pd.read_csv(IN / "main_sample.csv", encoding='utf8')

data_clean = data_clean.drop([0, 1])

data_clean['LocationLatitude'] = data_clean['LocationLatitude'].astype(float)
data_clean['LocationLongitude'] = data_clean['LocationLongitude'].astype(float)



def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6371000  # meters
    rad1 = math.radians(lat1)  # φ, λ in radians φ1
    rad2 = math.radians(lat2) #φ2
    diff1 = math.radians(lat2 - lat1)
    diff2 = math.radians(lon2 - lon1)

    a = math.sin(diff1/2) * math.sin(diff1/2) + \
        math.cos(rad1) * math.cos(rad2) * \
        math.sin(diff2/2) * math.sin(diff2/2)
    
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))

    d = R * c  # in meters

    d = d / 1000
    d = round(d, 2)
    return d

def coal_prox_indicator(data_clean):

    #https://www.globaldata.com/data-insights/mining/india--five-largest-coal-mines-in-2090702/#:~:text=The%20five%20largest%20coal%20mines,mmtpa%20of%20ROM%20in%202021.
    coal_mines = {
        'Gevra OC Mine' : [22.337439, 82.546566],
        'Bhubaneswari OCP Mine' : [20.965332, 85.162714],
        'Dipka OC Project': [22.327948, 82.557578],
        'Kusmunda OC Mine' : [22.330294, 82.669243],
        'Lakhanpur OCP Mine' : [21.765555, 83.820422],
        'Jharia Coalfield' : [23.722714, 86.443399],
        'Giridih Coalfield' : [24.154031, 86.301516],
        'Bokaro Coalfield' : [23.801295, 85.866097],
        'Karanpura Coalfield' : [23.614122, 85.679237],
        'Talcher Coalfield' : [20.956791, 85.142703],
        'Ib Valley Coalfield' : [21.772643, 83.865977],
        'Mand-Raigarh Coalfield' : [22.098944, 83.120011],
        'Korba Coalfield' : [22.318300, 82.735459],
        'Raniganj' : [23.679794, 87.113957],
        'Singrauli' : [24.142592, 82.654972],
        'Satpura' : [22.117582, 78.156525],
        'Telangana' : [18.708977, 79.531696],
        'Telangana 2' : [17.950403, 80.777345],
        'Telangana 3' : [17.200932, 80.798911],
        'Telangana 4' : [17.458652, 80.654307],
        'TamilNadu' : [11.531606, 79.464314],
        'Maharashtra' : [18.642455, 79.575385],
        'Madhya Pradesh' : [22.275820, 78.957188],
        'Assam' : [27.278341, 95.741360],
        'Jharkand' : [23.768009, 86.397751],
        'Giridih' : [24.153271, 86.302280],
        'Namchik' : [27.396511, 96.016445],
    }

    #https://www.power-technology.com/features/feature-the-top-10-biggest-thermal-power-plants-in-india/?cf-view
    coal_plants = {
        'Mundra Thermal Power Station, Gujarat' : [22.824412, 69.547972],
        'Mundra Ultra Mega Power Plant, Gujarat' : [22.815815, 69.525399],
        'Sasan Ultra Mega Power Plant, Madhya Pradesh' : [23.983091, 82.618795],
        'Tiroda Thermal Power Plant, Maharashtra' : [21.413246, 79.966920],
        'Talcher Super Thermal Power Station, Odisha' : [21.096245, 85.073762],
        'Rihand Thermal Power Station, Uttar Pradesh' : [24.027386, 82.790051],
        'Sipat Thermal Power Plant, Chhattisgarh' : [22.136485, 82.289925],
        'Chandrapur Super Thermal Power Station, Maharashtra' : [20.005170, 79.293512],
        'NTPC Dadri, Uttar Pradesh' : [28.597505, 77.607886],
    }

    for name, coordinates in coal_mines.items():
        data_clean[f'coal_mine_prox_{name}'] = data_clean.apply(lambda row: calculate_distance(row['LocationLatitude'], row['LocationLongitude'], coordinates[0], coordinates[1]), axis=1)

    for name, coordinates in coal_plants.items():
        data_clean[f'coal_plant_prox_{name}'] = data_clean.apply(lambda row: calculate_distance(row['LocationLatitude'], row['LocationLongitude'], coordinates[0], coordinates[1]), axis=1)


    matching_columns = [col for col in data_clean.columns if col.startswith("coal_mine")]  #or col.startswith("coal_plant")

    # Check if any value in the matching columns is smaller than 100
    data_clean['coal_prox'] = data_clean[matching_columns].lt(50).any(axis=1).astype(int)

    return data_clean

df = coal_prox_indicator(data_clean)

In [19]:
data_clean = pd.read_csv(IN / "main_sample.csv", encoding='utf8')

data_clean = data_clean.drop([0, 1])